# Read Data

In [ ]:
PATH = './'

In [135]:
import pandas as pd
import os
import optuna
train_df = pd.read_csv(f'{PATH}final_log_df_train.csv')
dev_df = pd.read_csv(f'{PATH}final_log_df_dev.csv')
train_column_rename = {}
for col in train_df.columns:
    new_column_name = col.replace('train', '')
    train_column_rename[col] = new_column_name
train_df.rename(columns=train_column_rename, inplace=True)
train_df['tag'] = train_df['tag'].map({'T': 1, 'F':0})
dev_df['tag'] = dev_df['tag'].map({'T': 1, 'F':0})
dev_column_rename = {}
for col in dev_df.columns:
    new_column_name = col.replace('dev', '')
    dev_column_rename[col] = new_column_name
dev_df.rename(columns=dev_column_rename, inplace=True)

In [136]:
dev_df

,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=3+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,tag,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=8+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=16+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=32+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=60+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=75+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=100+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=115+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=130+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,...,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=16+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=32+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=60+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=75+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=100+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=115+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=130+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=150+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=175+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=240+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_
0,0.0,0,0.066667,0.142857,0.185185,0.263158,0.250000,0.234568,0.229947,0.244019,...,0.066667,0.066667,0.052632,0.056338,0.047120,0.045455,0.052632,0.060071,0.067073,0.071429
1,0.2,1,0.230769,0.391304,0.306122,0.379310,0.351351,0.315789,0.299435,0.333333,...,0.103448,0.066667,0.061947,0.102941,0.123596,0.116505,0.111111,0.111111,0.104101,0.116279
2,0.0,0,0.000000,0.000000,0.000000,0.008403,0.006711,0.036269,0.031390,0.031746,...,0.032258,0.015873,0.043478,0.056338,0.092896,0.090047,0.101695,0.098901,0.090343,0.098398
3,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.040724,0.044177,...,0.000000,0.015873,0.034483,0.034483,0.063830,0.069767,0.061224,0.067616,0.057402,0.073826
4,0.0,1,0.066667,0.103448,0.230769,0.276596,0.271186,0.265823,0.299435,0.306533,...,0.066667,0.122807,0.176471,0.181102,0.149425,0.138614,0.130435,0.132075,0.155116,0.156627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,1,0.066667,0.103448,0.122807,0.188119,0.181102,0.197605,0.204188,0.198157,...,0.185185,0.207547,0.153846,0.145038,0.169591,0.167513,0.176471,0.162791,0.178451,0.203008
996,0.0,0,0.000000,0.000000,0.015873,0.081081,0.094891,0.104972,0.121951,0.155556,...,0.000000,0.000000,0.025641,0.034722,0.025773,0.026906,0.031873,0.027491,0.026471,0.036797
997,0

In [137]:
y_train = train_df.tag
train_df.drop(columns='tag', inplace=True)

y_dev = dev_df.tag
dev_df.drop(columns='tag', inplace=True)

In [138]:
dev_df

,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=3+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=8+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=16+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=32+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=60+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=75+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=100+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=115+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=130+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=150+<mask>-(and-also-T)-2ltr1f_topk500_fixspacesTruelemmIoUforLog+0.0++0.0+_,...,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=16+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=32+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=60+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=75+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=100+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=115+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=130+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=150+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=175+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_,xlmr-baseline.metric=IoU.post_process=post_process_lemm.kmasks_1=240+T-or-<mask><mask><mask>-2ltr3f_topk500_fixspacesTruelemmIoUforLog+0.0+_
0,0.0,0.066667,0.142857,0.185185,0.263158,0.250000,0.234568,0.229947,0.244019,0.244813,...,0.066667,0.066667,0.052632,0.056338,0.047120,0.045455,0.052632,0.060071,0.067073,0.071429
1,0.2,0.230769,0.391304,0.306122,0.379310,0.351351,0.315789,0.299435,0.333333,0.315789,...,0.103448,0.066667,0.061947,0.102941,0.123596,0.116505,0.111111,0.111111,0.104101,0.116279
2,0.0,0.000000,0.000000,0.000000,0.008403,0.006711,0.036269,0.031390,0.031746,0.038062,...,0.032258,0.015873,0.043478,0.056338,0.092896,0.090047,0.101695,0.098901,0.090343,0.098398
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.040724,0.044177,0.052632,...,0.000000,0.015873,0.034483,0.034483,0.063830,0.069767,0.061224,0.067616,0.057402,0.073826
4,0.0,0.066667,0.103448,0.230769,0.276596,0.271186,0.265823,0.299435,0.306533,0.321586,...,0.066667,0.122807,0.176471,0.181102,0.149425,0.138614,0.130435,0.132075,0.155116,0.156627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.066667,0.103448,0.122807,0.188119,0.181102,0.197605,0.204188,0.198157,0.204819,...,0.185185,0.207547,0.153846,0.145038,0.169591,0.167513,0.176471,0.162791,0.178451,0.203008
9

#  Inference

In [139]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import numpy as np
def fit_predict(n_splits, params, x_train, y_train, x_test):
    
    oof = np.zeros(x_train.shape[0])
    
    y_preds = []
    
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    for train_idx, valid_idx in cv.split(x_train, y_train):
        
        x_train_train = train_df.iloc[train_idx]
        y_train_train = y_train.iloc[train_idx]
        x_train_valid = train_df.iloc[valid_idx]
        y_train_valid = y_train.iloc[valid_idx]

        lgb_train = lgb.Dataset(data=x_train_train.astype('float32'), label=y_train_train.astype('float32'))
        lgb_valid = lgb.Dataset(data=x_train_valid.astype('float32'), label=y_train_valid.astype('float32'))

        estimator = lgb.train(params, lgb_train, 10000, valid_sets=lgb_valid,
                              early_stopping_rounds=25, verbose_eval=0)

        oof_part = estimator.predict(x_train_valid, num_iteration=estimator.best_iteration)
        oof[valid_idx] = oof_part
        
        if x_test is not None:
            y_part = estimator.predict(x_test, num_iteration=estimator.best_iteration)
            y_preds.append(y_part)
        
    score = log_loss(y_train, oof)
    print('LogLoss Score:', score)
    
    y_pred = np.mean(y_preds, axis=0)
    
    return y_pred, oof, score

In [145]:
score_list = []
all_params = []

In [146]:
def objective(trial):
    
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'boost_from_average': True,
        'num_threads': 4,
        'random_state': 42,
        
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 200),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'bagging_fraction': trial.suggest_loguniform('bagging_fraction', .5, .99),
        'feature_fraction': trial.suggest_loguniform('feature_fraction', .5, .99),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 2),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 2)
    }
    
    scores = []
    _, _, score = fit_predict(5, params, train_df, y_train, None)
    scores.append(score)
    score_list.append(np.mean(scores))
    all_params.append(params)
    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2021-05-22 07:18:44,236] A new study created in memory with name: no-name-35277479-f16e-4a26-b8df-893b731df57a
/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-05-22 07:18:55,312] Trial 0 finished with value: 0.42499544218305074 and parameters: {'num_leaves': 863, 'min_data_in_leaf': 47, 'min_child_weight': 0.008948562794658634, 'max_depth': 21, 'bagging_fraction': 0.5100433783698829, 'feature_fraction': 0.6733304408729441, 'lambda_l1': 0.7849678148717756, 'lambda_l2': 0.5625608497268687}. Best is trial 0 with value: 0.42499544218305074.


LogLoss Score: 0.42499544218305074


/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-05-22 07:19:01,858] Trial 1 finished with value: 0.4209158229260208 and parameters: {'num_leaves': 236, 'min_data_in_leaf': 142, 'min_child_weight': 0.03043705788970173, 'max_depth': 73, 'bagging_fraction': 0.5298156612635471, 'feature_fraction': 0.5609156720518224, 'lambda_l1': 1.7681407832093323, 'lambda_l2': 0.2031382304472772}. Best is trial 1 with value: 0.4209158229260208.


LogLoss Score: 0.4209158229260208


/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-05-22 07:19:09,630] Trial 2 finished with value: 0.4230356916895638 and parameters: {'num_leaves': 444, 'min_data_in_leaf': 158, 'min_child_weight': 0.016431597317006285, 'max_depth': 74, 'bagging_fraction': 0.6643916594134726, 'feature_fraction': 0.7171898782228667, 'lambda_l1': 0.8306848600431477, 'lambda_l2': 1.3030477966732323}. Best is trial 1 with value: 0.4209158229260208.


...

LogLoss Score: 0.42361210757737544


/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-05-22 07:35:40,642] Trial 73 finished with value: 0.4225432454025887 and parameters: {'num_leaves': 387, 'min_data_in_leaf': 126, 'min_child_weight': 0.002705961804889859, 'max_depth': 13, 'bagging_fraction': 0.523843290670527, 'feature_fraction': 0.5878077967934784, 'lambda_l1': 1.0836187894436105, 'lambda_l2': 0.29555291988922033}. Best is trial 21 with value: 0.4206535492796133.


LogLoss Score: 0.4225432454025887


/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-05-22 07:36:05,164] Trial 74 finished with value: 0.42471668389496875 and parameters: {'num_leaves': 321, 'min_data_in_leaf': 77, 'min_child_weight': 0.011689973993629142, 'max_depth': 17, 'bagging_fraction': 0.552390943404194, 'feature_fraction': 0.6033136020068446, 'lambda_l1': 1.305622405861268, 'lambda_l2': 0.3332610788225839}. Best is trial 21 with value: 0.4206535492796133.


LogLoss Score: 0.42471668389496875


In [ ]:
study.best_trial 

In [ ]:

params={'num_leaves': 560, 'min_data_in_leaf': 189, 'min_child_weight': 0.0024699349965224245, 'max_depth': 26, 'bagging_fraction': 0.9753023991207269, 'feature_fraction': 0.7137858794613544, 'lambda_l1': 0.5330703499296066, 'lambda_l2': 0.12028486914560135}

lgb_train = lgb.Dataset(data=train_df.astype('float32'), label=y_train.astype('float32'))
lgb_valid = lgb.Dataset(data=dev_df.astype('float32'), label=y_dev.astype('float32'))

estimator = lgb.train(params, lgb_train, 800, verbose_eval=0)

lgbm_probs = estimator.predict(dev_df)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_dev,check)


In [73]:
accuracy_score(y_dev, (lgbm_probs > 0.5).astype(int))

0.843

### Bagging LGBM

In [34]:
classifiers = []

n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_idx, valid_idx in cv.split(train_df, y_train):
    params={'num_leaves': 560, 'min_data_in_leaf': 189, 'min_child_weight': 0.0024699349965224245, \
            'max_depth': 26, 'bagging_fraction': 0.9753023991207269, \
            'feature_fraction': 0.7137858794613544, 'lambda_l1': 0.5330703499296066, \
            'lambda_l2': 0.12028486914560135, \
           'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'boost_from_average': True,
    'num_threads': 4,
    'random_state': 42} # minimize
    x_train_train = train_df.iloc[train_idx]
    y_train_train = y_train.iloc[train_idx]
    x_train_valid = train_df.iloc[valid_idx]
    y_train_valid = y_train.iloc[valid_idx]

    lgb_train = lgb.Dataset(data=x_train_train.astype('float32'), label=y_train_train.astype('float32'))
    lgb_valid = lgb.Dataset(data=x_train_valid.astype('float32'), label=y_train_valid.astype('float32'))

    estimator = lgb.train(params, lgb_train, 10000, valid_sets=lgb_valid,
                          early_stopping_rounds=25, verbose_eval=0)
    classifiers.append(estimator)

In [35]:

preds = []
for clf in classifiers:
    y_part = estimator.predict(dev_df, num_iteration=estimator.best_iteration)
    preds.append(y_part)
preds = np.mean(preds, axis=0)

check = (preds > 0.5).astype(int)
accuracy_score(y_dev, check)

0.839

# Random Forest

In [37]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn import linear_model
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection
def fit_predict(n_splits, params, x_train, y_train, x_test):
    
    oof = np.zeros(x_train.shape[0])
    
    y_preds = []
    
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7)
    for train_idx, valid_idx in cv.split(x_train, y_train):
        
        x_train_train = train_df.iloc[train_idx]
        y_train_train = y_train.iloc[train_idx]
        x_train_valid = train_df.iloc[valid_idx]
        y_train_valid = y_train.iloc[valid_idx]

        estimator = ensemble.RandomForestClassifier(**params)
        estimator.fit(x_train_train, y_train_train)
        oof_part = estimator.predict(x_train_valid)
        oof[valid_idx] = oof_part
        
        if x_test is not None:
            y_part = estimator.predict(x_test)
            y_preds.append(y_part)
        
    score = accuracy_score(y_train, oof)
    print('Accuracy:', score)
    
    y_pred = np.mean(y_preds, axis=0)
    
    return y_pred, oof, score

In [38]:
def objective(trial):
    # Categorical parameter
    params = {
    'criterion': trial.suggest_categorical('rf_criterion', ['gini']),

   'n_estimators': trial.suggest_int('n_estimators', 200, 400),
            'max_depth': trial.suggest_int('max_depth', 11, 18),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 15)
    }
    scores = []
    _, _, score = fit_predict(5, params, train_df, y_train, None)
    scores.append(score)
    
    return np.mean(scores)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)


[I 2021-05-22 05:13:39,480] A new study created in memory with name: no-name-c34a320d-112d-46ee-bb0f-ae086b944f2e
/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/vsevolod/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-05-22 05:14:39,809] Trial 0 finished with value: 0.804625 and parameters: {'rf_criterion': 'gini', 'n_estimators': 341, 'max_depth': 15, 'min_samples_split': 8, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.804625.


Accuracy: 0.804625


KeyboardInterrupt: 

In [ ]:
study.best_params

In [119]:


rf_pars = {'criterion': 'gini',
 'n_estimators': 342,
 'max_depth': 17,
 'min_samples_split': 6,
 'min_samples_leaf': 1}


In [120]:
estimator = ensemble.RandomForestClassifier(**rf_pars)
estimator.fit(train_df, y_train)

rf_preds = estimator.predict(dev_df)
accuracy_score(y_dev, rf_preds)

0.844

In [121]:
rf_preds_proba = estimator.predict_proba(dev_df)

In [124]:
accuracy_score(y_dev,(((lgbm_probs * 0.822 + rf_preds_proba[:, 1] * 0.8125) \
                       / (0.822 + 0.8125)) > 0.5).astype(int))

0.853

#  Logistic Regression

In [59]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn import linear_model
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection
def fit_predict(n_splits, params, x_train, y_train, x_test):
    
    oof = np.zeros(x_train.shape[0])
    
    y_preds = []
    
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7)
    for train_idx, valid_idx in cv.split(x_train, y_train):
        
        x_train_train = train_df.iloc[train_idx]
        y_train_train = y_train.iloc[train_idx]
        x_train_valid = train_df.iloc[valid_idx]
        y_train_valid = y_train.iloc[valid_idx]

        estimator = linear_model.LogisticRegression(**params)
        estimator.fit(x_train_train, y_train_train)
        oof_part = estimator.predict(x_train_valid)
        oof[valid_idx] = oof_part
        
        if x_test is not None:
            y_part = estimator.predict(x_test)
            y_preds.append(y_part)
        
    score = accuracy_score(y_train, oof)
    print('Accuracy:', score)
    
    y_pred = np.mean(y_preds, axis=0)
    
    return y_pred, oof, score




In [60]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()

train_df_scaled = scaler.fit_transform(train_df[train_df.columns])
dev_df_scaled = scaler.transform(dev_df[dev_df.columns])

In [61]:

def objective(trial):
    # Categorical parameter
    params = {
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'C' : trial.suggest_float("C", 1e-10, 1e10, log=True),
        'solver': trial.suggest_categorical('solver', ['saga']),
        'max_iter': trial.suggest_int('max_iter', 100, 500)
    }
    scores = []
    _, _, score = fit_predict(5, params, train_df_scaled, y_train, None)
    scores.append(score)
    
    return np.mean(scores)

In [62]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2021-05-22 05:39:29,698] A new study created in memory with name: no-name-a737f855-461c-442c-ae58-b82cf60ef655
/home/vsevolod/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/vsevolod/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/vsevolod/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/vsevolod/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/vsevolod/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/vsevolod/anacon

Accuracy: 0.786


KeyboardInterrupt: 

In [ ]:
study.best_score
study.best_trial.value

In [ ]:
lr_params = {'penalty': 'l2', 'C': 1.146016100950042, 'solver': 'saga', 'max_iter': 254}

In [128]:

estimator = linear_model.LogisticRegression(**lr_params)
estimator.fit(train_df_scaled, y_train)
lr_preds = estimator.predict(dev_df_scaled)
lr_preds_proba = estimator.predict_proba(dev_df_scaled)
accuracy_score(y_dev, lr_preds)

/home/vsevolod/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



0.828

In [130]:
roc_auc_score(y_dev, lr_preds_proba[:, 1])

0.9123479999999999

# Blend all

In [131]:
lr_score = 0.74
lgbm_score = 0.822
rf_score = 0.8125

In [132]:
accuracy_score(y_dev, (((lr_preds_proba[:, 1] * lr_score + rf_preds_proba[:, 1] * rf_score + lgbm_probs * lgbm_score) \
                        / (lr_score + rf_score + lgbm_score)) > 0.5).astype(int))

0.845

In [133]:
# Blend Best

In [134]:
accuracy_score(y_dev,(((lgbm_probs * 0.822 + rf_preds_proba[:, 1] * 0.8125) \
                       / (0.822 + 0.8125)) > 0.5).astype(int))

0.853